In [14]:
#group members
#ES19BTECH11003(Hemanth K)              CS19BTECH11009(Naveen)
#!/usr/bin/env python
# coding: utf-8

# Input Format :
#     give all the constraints in less than equal to form 
#     for e.g if the question is

#        maximize 		(25x+20y) 
#     	  subject to    20x+12y <=2000
#                       5x+5y<=540

# There is no need to give trivial constraints as input such as x>=0, y>=0 as we have already written code to add them automatically as a constraints.
#                 
# then input should be     
# number of rows = 2    
# number of columns =2
# Entries of A     20 12 5 5 
# Entries of B     2000 540 
# Entries of C     25 20

import sys
import numpy as np
from scipy.linalg import solve
from scipy.linalg import null_space
from sympy import Matrix 

def check_constraints(A, B, x):
	delta = pow(10, -8)
	B = B + delta
	result = np.dot(A, x)
	test = np.less_equal(result, B)
	Bbar = (test == False)
	if test[Bbar].size <= 0:
		return True
	else:
		return False 



def check_constraints_y(A, B, y):
	delta = pow(10, -8)
	result = np.dot(A, y)
	difference = np.subtract(B, result)
	test = (difference >= delta)
	Bbar = (test == False)
	if test[Bbar].size <= 0:
		return True
	else:
		return False 

def tight_constraint(A, B, x, nulls, cols, alpha):
	delta = pow(10,-8)
	j, i = 0, 0
	while(j < cols):
		tmp = x
		x = x + (alpha*nulls)
		i = i + 1
		check = check_constraints(A, B, x)
		if(i >= 9999):
			x = np.NaN
			break
		if check:
			a = np.asarray(A @ x)
			difference = np.subtract(B, a)
			difference = abs(difference)
			test = difference < delta
			Bbar = (test==True)    
			j = test[Bbar].size
		else:
			alpha = alpha/10
			x = tmp
	return x

def border_point_calculation(A, B, x, cols): 
	step = np.ones((cols, 1))
	check = check_constraints(A, B, x)
	tmp = x
	while check:
		tmp = x
		x = np.add(x, step)
		check = check_constraints(A, B, x)

	y = tmp
	check = True
	while True:
		step = step/10.0
		check = check_constraints_y(A, B, y)
		if check == False:
			break 
		test = True
		while test:
			x = y
			y = np.add(y, step)
			test = check_constraints(A, B, y)
		y = x
	return x


# In[52]:


def main():
	rows = int(input("Please enter the number of rows in integer format : "))
	cols = int(input("Please enter the number of columns in integer format : "))

	# Enter the number of rows and columns and corresponding entries of matrix A, B and C

	print("Enter the elements of matrix A in a single line (separated by space): ")
	elements = list(map(float, input().split())) 
	A = np.array(elements).reshape(rows, cols)

	Abar = -1*np.identity(cols)
	A = np.concatenate((A, Abar))
	print(A)
	
	print("Enter the entries of matrix B in a single line (separated by space): ")
	elements = list(map(float, input().split())) 
	B = np.array(elements).reshape(rows, 1)

	const_col = np.zeros((cols, 1))
	B = np.vstack([B, const_col])
	print(B)
	
	print("Enter the entries of matrix C in a single line (separated by space): ")
	elements = list(map(float, input().split())) 
	C = np.array(elements).reshape(cols, 1)
	print(C)
    
	rows += cols
	f1, f, min_val = 0, 0, np.min(B)	        # check whether that any value of B is negative
							
	delta = pow(10, -8)
	if min_val < 0:								# if any value of B is negative then add an extra row and column by multipying it by -1 and maximize the value of z which is x[n,0]
		Abar = np.copy(A)
		const_row = np.zeros((1, cols))
		Abar = np.vstack([Abar, const_row])
		const_col = np.ones((1 + rows, 1))
		const_col[rows, 0] = -1
		Abar = np.hstack([Abar, const_col])
		
		Bbar = np.copy(B)
		Bbar = np.vstack([Bbar, -1 * min_val])
		Bbar.reshape(1, 1 + rows)

		if rows > cols:
			x = np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(Abar), Abar)), np.transpose(Abar)), Bbar)  #left-inverse
		elif rows < cols:
			x = np.dot(np.dot(np.transpose(Abar), np.linalg.inv(np.dot(Abar, np.transpose(Abar)))), Bbar)  #right-inverse

		z = x[cols, 0]
		if z < 0:						                # if value  of z is negative then the solution is infeasible.
			print("Infeasible Solution")
			return
		else:								            # if value of z is non negative then either the solution exists or it is unbounded.
			x = np.delete(x, cols)
			x = x.reshape(cols, 1)
			x = border_point_calculation(A, B, x, cols)		# find any point in bounded region and move in any direction to satisfy the constraints.

			bdash = A.dot(x)
			difference = np.subtract(B, bdash)
			test = (difference < delta)
			Bbar = (test == True)
			v = 0
			c = test[Bbar].size
			_tight_constraint = np.zeros((1, cols))
			for i in range(rows):
				if(test[i]==True):
					r, v = A[i], v+1
					_tight_constraint=np.vstack([_tight_constraint, r])
					
			_tight_constraint = np.delete(_tight_constraint, (0), axis=0)
			if v != n:
				f = 1
				nulls = null_space(_tight_constraint)
	else:
		x = np.zeros((cols, 1))
		f1 = 1

	iteration, alpha = 0, 1
	while True:
		iteration = iteration + 1
		if iteration == 100000:					# if iteration is exeeding 100000 then the solution is unbounded
			print("Unbounded solution")
			break
		if f == 1 and f1 == 0:
			x = tight_constraint(A, B, x, nulls, cols, alpha)
		if(f == 1 and np.any(np.isnan(x))):
			print("Unbounded soultion")
			break
		f, f1 = 1, 0

		Q = np.zeros((1,1))
		T = np.zeros((1, cols))
		
		bdash =np.asarray(A @ x)
		difference = np.subtract(B, bdash)
		check = (difference < delta)
		Bbar = (check==True)    
		c=check[Bbar].size
		count = 0
		for i in range(rows):
			if(check[i] == True):
				count = count + 1
				k, l = A[i], B[i]
				Q = np.vstack([Q, l])
				T = np.vstack([T, k])
			

		T = np.delete(T, (0), axis=0)
		Q = np.delete(Q, (0), axis=0)

		if count > cols:
			print("It is a degenerate case")
			Q = Q[0:cols, :]
			T = T[0:cols, :]
			

		T_inverse = np.linalg.inv(T)	
		y = C.transpose() @ T_inverse

		if np.min(y) >= 0:									# if all values are greater than 0 then it is feasible point
			print("The optimal point is :")
			x = np.linalg.solve(T, Q)
			print(x)
			print("The optimum cost is :")
			print(x.transpose() @ C)
			break
		else:
			k = 0
			index = 0
			for i in range (cols):           
				if y[0,i] < 0:
					index = k
					break
				k = k + 1

			nulls = T_inverse[:, [index]]					
			nulls = -1*nulls
			x = np.linalg.solve(T, Q)

if __name__ == "__main__":
	main()

Please enter the number of rows in integer format : 4
Please enter the number of columns in integer format : 2
Enter the elements of matrix A in a single line (separated by space): 
-1 0 0 -1 -1 -1 0 1
[[-1.  0.]
 [ 0. -1.]
 [-1. -1.]
 [ 0.  1.]
 [-1. -0.]
 [-0. -1.]]
Enter the entries of matrix B in a single line (separated by space): 
-1 -1 -4 3
[[-1.]
 [-1.]
 [-4.]
 [ 3.]
 [ 0.]
 [ 0.]]
Enter the entries of matrix C in a single line (separated by space): 
-2 -3
[[-2.]
 [-3.]]
Infeasible Solution
